In [ ]:
import numpy as np
import os
from os import urandom

In [ ]:
W=16
ALPHA=7
BETA=2
MASK_VAL = 2 ** W - 1;

In [ ]:
def rotate_left(x,k):
    return(((x << k) & MASK_VAL) | (x >> (W - k)));

def rotate_right(x,k):
    return((x >> k) | ((x << (W - k)) & MASK_VAL));

def enc_one_round(p, k):
    cL, cR = p[0], p[1];
    cL = rotate_right(cL, ALPHA);
    cL = (cL + cR) & MASK_VAL;
    cL = cL ^ k;
    cR = rotate_left(cR, BETA);
    cR = cR ^ cL;
    return(cL,cR);

def dec_one_round(c,k):
    cL, cR = c[0], c[1];
    cR = cR ^ cL;
    cR = rotate_right(cR, BETA);
    cL = cL ^ k;
    cL = (cL - cR) & MASK_VAL;
    cL = rotate_left(cL, ALPHA);
    return(cL, cR);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);


In [ ]:
def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

In [ ]:
#main
key = (0x1918,0x1110,0x0908,0x0100)
pt = (0x6574, 0x694c)
ks = expand_key(key, 22)
ct = encrypt(pt, ks)

if (ct == (0xa868, 0x42f2)):
    print("Testvector is verified.")
else:
    print("Testvector is not verified.")

#decrypt
pt1=decrypt(ct,ks)
pt1=(pt1[0]<<16)|pt[1]
print(f"Original plaintext: {hex(pt[0]<<16|pt[1])}\nDecrpyted plaintext: {hex(pt1)}")

Testvector verified.
original plaintext: 0x6574694c
decrpyted plaintext: 0x6574694c


In [ ]:
#takes input array of ciphertext pairs

def convert_to_binary(arr):
  Y = np.zeros((4 * W,len(arr[0])),dtype=np.uint8);
  for i in range(4 * W):
    index = i // W;
    offset = W - (i % W) - 1;
    Y[i] = (arr[index] >> offset) & 1;
  Y = Y.transpose();
  return(Y);

In [ ]:
#train data generator
np.random.seed(42)
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

In [ ]:
X,Y=make_train_data(100,22)
X.shape

(100, 64)

In [ ]:
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels and keys
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);

  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);

  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);

  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random (y=0)
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);